In [1]:
from api import MovieApi 

In [2]:
session = MovieApi()
movies = session.get_top_rated_movies().json()

In [3]:
genres = movies['genres']
# new_genre_columns = [str(g['id']) + '_' + str(g['name']) for g in genres] # checkout list comprehesion https://www.digitalocean.com/community/tutorials/understanding-list-comprehensions-in-python-3
new_genre_columns = [str(g['id']) for g in genres] 
new_genre_columns

['28',
 '12',
 '16',
 '35',
 '80',
 '99',
 '18',
 '10751',
 '14',
 '36',
 '27',
 '10402',
 '9648',
 '10749',
 '878',
 '10770',
 '53',
 '10752',
 '37']

In [4]:
import pandas as pd
top_movies = pd.read_csv('./top_rated_movies33.csv').drop(columns= ['Unnamed: 0'],axis=1)
top_movies.shape

(10000, 14)

### Clean up grenre column 

In [5]:
# remove spaces in genre names 
for i, val in enumerate(new_genre_columns):
    new_genre_columns[i] = val.replace(' ','_')
new_genre_columns



['28',
 '12',
 '16',
 '35',
 '80',
 '99',
 '18',
 '10751',
 '14',
 '36',
 '27',
 '10402',
 '9648',
 '10749',
 '878',
 '10770',
 '53',
 '10752',
 '37']

In [6]:
# add 
for g in new_genre_columns:
    top_movies[g] = 0
top_movies.columns
top_movies.shape

(10000, 33)

In [7]:
top_movies.dtypes

adult                   bool
backdrop_path         object
genre_ids             object
movie_id               int64
original_language     object
original_title        object
overview              object
popularity           float64
poster_path           object
release_date          object
title                 object
video                   bool
vote_average         float64
vote_count             int64
28                     int64
12                     int64
16                     int64
35                     int64
80                     int64
99                     int64
18                     int64
10751                  int64
14                     int64
36                     int64
27                     int64
10402                  int64
9648                   int64
10749                  int64
878                    int64
10770                  int64
53                     int64
10752                  int64
37                     int64
dtype: object

In [8]:
top_movies['genre_ids'] = top_movies['genre_ids'].astype(str).apply(lambda a : a.strip(']').strip('['))
top_movies.tail()

,adult,backdrop_path,genre_ids,movie_id,original_language,original_title,overview,popularity,poster_path,release_date,...,36,27,10402,9648,10749,878,10770,53,10752,37
9995,False,/zhLcgqoIDvxNZPtvQ1PYloGalXs.jpg,"53, 27, 9648",61012,en,Silent Hill: Revelation 3D,Heather Mason and her father have been on the ...,41.568,/xBC7bHZ5UglTw4LMd2lTSgXvQXl.jpg,2012-10-10,...,0,0,0,0,0,0,0,0,0,0
9996,False,/g9ekJY8zNxQ9J3cRYQHzRwiObuq.jpg,"10749, 35",11812,en,The Sweetest Thing,Christina's love life is stuck in neutral. Aft...,15.915,/qLysz4B5YmiDq52Op0zq4eD75LC.jpg,2002-04-12,...,0,0,0,0,0,0,0,0,0,0
9997,False,/eqZ5t82asfprFcNS56eVruNT5G5.jpg,"10751, 35",10481,en,102 Dalmatians,Get ready for a howling good time as an all ne...,18.685,/dSxnIika9yWwTvEbpsmoGdeh65E.jpg,2000-11-21,...,0,0,0,0,0,0,0,0,0,0
9998,False,/gdF4eUHLeEizqTdSGoqfWPIdDX6.jpg,35,450179,fr,Problemos,"When a pandemic strikes the world, Victor and ...",6.521,/12KzS7zZ2Kj4p556lvR7UtJsQoT.jpg,2017-05-10,...,0,0,0,0,0,0,0,0,0,0
9999,False,/1u7TuvR5mSUz2xRQhz01i75IwdT.jpg,"80, 53, 28",201088,en,Blackhat,"Nicholas Hathaway, a furloughed convict, and h...",32.736,/7zu6HPnKPjNA7NuR4DGRdaH0PdX.jpg,2015-01-13,...,0,0,0,0,0,0,0,0,0,0


### Genre column clean up end ------------------------------

were you left off update:

you got the genre column data cleaned up enough to where you can loop through each row, extract the genre data from the original row and place it in the correct new column

you are trying to visualize data BUTTTTTTT the genre column has multiple ids per movie and its stored a list thats been turned into a string and you cant undo that ....so far

... you did that girl :snaps:



In [10]:
top_movies.tail(3)

,adult,backdrop_path,genre_ids,movie_id,original_language,original_title,overview,popularity,poster_path,release_date,...,36,27,10402,9648,10749,878,10770,53,10752,37
9997,False,/eqZ5t82asfprFcNS56eVruNT5G5.jpg,"10751, 35",10481,en,102 Dalmatians,Get ready for a howling good time as an all ne...,18.685,/dSxnIika9yWwTvEbpsmoGdeh65E.jpg,2000-11-21,...,0,0,0,0,0,0,0,0,0,0
9998,False,/gdF4eUHLeEizqTdSGoqfWPIdDX6.jpg,35,450179,fr,Problemos,"When a pandemic strikes the world, Victor and ...",6.521,/12KzS7zZ2Kj4p556lvR7UtJsQoT.jpg,2017-05-10,...,0,0,0,0,0,0,0,0,0,0
9999,False,/1u7TuvR5mSUz2xRQhz01i75IwdT.jpg,"80, 53, 28",201088,en,Blackhat,"Nicholas Hathaway, a furloughed convict, and h...",32.736,/7zu6HPnKPjNA7NuR4DGRdaH0PdX.jpg,2015-01-13,...,0,0,0,0,0,0,0,0,0,0


In [72]:
type('genre_items') == str

baby_df = top_movies.head(6)
baby_df['genre_ids'].to_str

In [93]:
# for genre in baby_df['genre_ids']:
#     print(genre)
test = top_movies.tail(3)

for row in range(test.shape[0]):
    print(test.iloc[[row]])

# for row in test:
#     print(test.iloc[[row]])

      adult                     backdrop_path  genre_ids  movie_id  \
9997  False  /eqZ5t82asfprFcNS56eVruNT5G5.jpg  10751, 35     10481   

     original_language  original_title  \
9997                en  102 Dalmatians   

                                               overview  popularity  \
9997  Get ready for a howling good time as an all ne...      18.685   

                           poster_path release_date  ... 36  27  10402  9648  \
9997  /dSxnIika9yWwTvEbpsmoGdeh65E.jpg   2000-11-21  ...  0   0      0     0   

      10749  878  10770  53  10752  37  
9997      0    0      0   0      0   0  

[1 rows x 33 columns]
      adult                     backdrop_path genre_ids  movie_id  \
9998  False  /gdF4eUHLeEizqTdSGoqfWPIdDX6.jpg        35    450179   

     original_language original_title  \
9998                fr      Problemos   

                                               overview  popularity  \
9998  When a pandemic strikes the world, Victor and ...       6.521   



In [92]:
baby_df.loc[baby_df['genre_ids'] == '35, 14']

,adult,backdrop_path,genre_ids,movie_id,original_language,original_title,overview,popularity,poster_path,release_date,...,36,27,10402,9648,10749,878,10770,53,10752,37
2,False,/ejniJnlOdtSgtbh8D7u2RxT6Uli.jpg,"35, 14",772071,es,Cuando Sea Joven,70-year-old Malena gets a second chance at lif...,59.051,/6gIJuFHh5Lj4dNaPG3TzIMl7L68.jpg,2022-09-14,...,0,0,0,0,0,0,0,0,0,0


In [68]:
def dosomething(genre_items):
    df = baby_df
    if type(genre_items) == str:
        df.loc[df['genre_ids'] == genre_items]['genre_ids']

In [69]:
# update_genre_column(top_movies.head(2),'36')
[dosomething(genre) for genre in baby_df['genre_ids']]

0    18, 80
1    18, 80
3    18, 80
Name: genre_ids, dtype: object
0    18, 80
1    18, 80
3    18, 80
Name: genre_ids, dtype: object
2    35, 14
Name: genre_ids, dtype: object
0    18, 80
1    18, 80
3    18, 80
Name: genre_ids, dtype: object
4    18, 36, 10752
Name: genre_ids, dtype: object
5    35, 18, 10749
Name: genre_ids, dtype: object


[None, None, None, None, None, None]

In [ ]:
# [ foo(x) for x in df['x'] ]
# If multiple columns are needed, then zip can be used to make a list of tuples.
# def list_impl(df):
#   cutoff_date = datetime.date.today() + datetime.timedelta(days=2)
#   return pd.Series([
#     eisenhower_action(priority == 'HIGH', due_date <= cutoff_date)
#     for (priority, due_date) in zip(df['priority'], df['due_date'])
#   ])
# This also threw a surprise. It took only 78.4 milliseconds, even better than itertuples!
# %timeit data_sample['action_list'] = list_impl(data_sample)
# 10 loops, best of 5: 78.4 ms per loop